# Association Analysis

It is an unsupervised learning tool that looks for hidden patterns so there is limited need for data prep and feature engineering. It is a good start for certain cases of data exploration and can point the way for a deeper dive into the data using other approaches. The python implementation in MLxtend should be very familiar to anyone that has exposure to scikit-learn and pandas.

Association rules are normally written like this: {Diapers} -> {Beer} which means that there is a strong relationship between customers that purchased diapers and also purchased beer in the same transaction.

In the above example, the {Diaper} is the antecedent and the {Beer} is the consequent. Both antecedents and consequents can have multiple items. In other words, {Diaper, Gum} -> {Beer, Chips} is a valid rule.

Support is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

Confidence is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

Lift is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

One final note, related to the data. This analysis requires that all the data for a transaction be included in 1 row and the items should be 1-hot encoded. The MLxtend documentation example is useful:

<img src ="basket.png">

In [70]:
%autosave 30

#import pandas as pd
#from mlxtend.frequent_patterns import apriori
#from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('spss.xlsx')
df.dtypes

Autosaving every 30 seconds


Person           int64
Household        int64
Adults           int64
Children         int64
Infants          int64
HHldr1           int64
HHldr2           int64
HHldr3           int64
HHldr4           int64
HHldr5           int64
HHldr6           int64
HHldr7           int64
HHldr8           int64
HHldr9           int64
HHldr10          int64
HHldr11          int64
HHldr12          int64
HHldr97          int64
tenure           int64
JoAccom          int64
LandLord         int64
furn             int64
edrooms          int64
PasSm            int64
NumSm            int64
Car              int64
NumCars          int64
FinOutc          int64
hhdtyp           int64
HHSize9          int64
                ...   
Health11         int64
HealthGP         int64
Hlthemp1         int64
Hlthemp2         int64
Hlthemp3         int64
Hlthemp4         int64
Hlthemp5         int64
Hlthemp6         int64
Hlthemp7         int64
Hlthemp8         int64
LAAss            int64
empnotaff        int64
UriIntro   

First, some of the descriptions have spaces that need to be removed. We’ll also drop the rows that don’t have invoice numbers and remove the credit transactions (those with invoice numbers containing C).

In [75]:
# replace -1 and -9 with blanks
import numpy as np
df.replace(-1, np.nan, inplace=True)
df.replace(-9, np.nan, inplace=True)

In [102]:
df.head(100)

,Person,Household,Adults,Children,Infants,HHldr1,HHldr2,HHldr3,HHldr4,HHldr5,...,Potass,Potassq,Creatin,Creatinq,wt_int,wt_nurse,wt_lood,wt_cotinine,wt_urine,wt_ltcp
0,148614101,1486141,2,0,0,1.0,1.0,0.0,0.0,0.0,...,101.0,NaN,23.9,NaN,0.882327,0.880031,0.000000,0.000000,0.806694,0.000000
1,148614102,1486141,2,0,0,1.0,1.0,0.0,0.0,0.0,...,21.0,NaN,3.3,NaN,0.846718,0.824500,0.000000,0.000000,0.751686,1.000575
2,148613101,1486131,1,1,0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.966176,0.000000,0.000000,0.000000,0.000000,1.000532
3,148613102,1486131,1,1,0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.971215,0.000000,0.000000,0.000000,0.000000,0.000000
4,148604101,1486041,2,0,0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,0.872772,0.000000,0.000000,0.000000,0.000000,0.971786
5,148604102,1486041,2,0,0,1.0,1.0,0.0,0.0,0.0,...,72.0,NaN,11.0,NaN,0.921173,1.012295,0.908878,0.000000,0.971389,0.000000
6,148610101,1486101,1,0,0,1.0,0.0,0.0,0.0,0.0,...,23.0,NaN,1.4,NaN,0.916397,0.981336,0.000000,0.000000,1.126971,0.765349
7,148609101,1486091,2,0,0,1.0,1.0,0.0,0.0,0.0,...,14.0,NaN,1.3,NaN,0.925327,0.926143,0.806917,0.000000,0.831684,0.000000
8,148611101,1486111,2,0,0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.023066,0.000000,0.000000,0.000000,0.000000,0.000000
9,148611102,1486111,2,0,0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.132797,0.000000,0.000000,0.000000,0.000000,1.276615


After the cleanup, we need to consolidate the items into 1 transaction per row with each product 1 hot encoded. For the sake of keeping the data set small, I’m only looking at sales for France.

There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and anything less the 0 is set to 0. This step will complete the one hot encoding of the data and remove the postage column (since that charge is not one we wish to explore):

In [126]:
def encode_units(x):
    if type(x) is float:
        if x <= 0.0:
            return 0.0
        if x > 0.0:
            return 1.0
    else: 
        return (x)

clean = df.applymap(encode_units)

In [127]:
clean.head()

,Person,Household,Adults,Children,Infants,HHldr1,HHldr2,HHldr3,HHldr4,HHldr5,...,Potass,Potassq,Creatin,Creatinq,wt_int,wt_nurse,wt_lood,wt_cotinine,wt_urine,wt_ltcp
0,148614101,1486141,2,0,0,1.0,1.0,0.0,0.0,0.0,...,1.0,NaN,1.0,NaN,1.0,1.0,0.0,0.0,1.0,0.0
1,148614102,1486141,2,0,0,1.0,1.0,0.0,0.0,0.0,...,1.0,NaN,1.0,NaN,1.0,1.0,0.0,0.0,1.0,1.0
2,148613101,1486131,1,1,0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0
3,148613102,1486131,1,1,0,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0
4,148604101,1486041,2,0,0,1.0,1.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,1.0


Now that the data is structured properly, we can generate frequent item sets that have a support of at least 7% (this number was chosen so that I could get enough useful examples):

In [7]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

The final step is to generate the rules with their corresponding support, confidence and lift

In [8]:
# Create the rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedants,consequents,support,confidence,lift
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.763158,7.478947
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.725000,7.478947
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.837838,8.642959
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.815789,8.642959
4,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE PINK),0.094388,0.783784,7.681081
5,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.725000,7.681081
6,(SPACEBOY LUNCH BOX),(DOLLY GIRL LUNCH BOX),0.125000,0.571429,5.743590
7,(DOLLY GIRL LUNCH BOX),(SPACEBOY LUNCH BOX),0.099490,0.717949,5.743590
8,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN SPACEBOY),0.168367,0.530303,3.849607
9,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN CIRCUS PARADE),0.137755,0.648148,3.849607


Now, the tricky part is figuring out what this tells us. For instance, we can see that there are quite a few rules with a high lift value which means that it occurs more frequently than would be expected given the number of transaction and product combinations. 

We can filter the dataframe using standard pandas code. In this case, look for a large lift (6) and high confidence (.8):

In [9]:
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedants,consequents,support,confidence,lift
2,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.837838,8.642959
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.815789,8.642959
16,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.800000,6.030769
18,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.888889,6.968889
19,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.960000,6.968889
20,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.812500,6.125000
21,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.975000,7.077778
22,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.975000,7.644000


In looking at the rules, it seems that the green and red alarm clocks are purchased together and the red paper cups, napkins and plates are purchased together in a manner that is higher than the overall probability would suggest.

At this point, you may want to look at how much opportunity there is to use the popularity of one product to drive sales of another. For instance, we can see that we sell 340 Green Alarm clocks but only 316 Red Alarm Clocks so maybe we can drive more Red Alarm Clock sales through recommendations?

In [10]:
basket['ALARM CLOCK BAKELIKE GREEN'].sum()

340.0

In [11]:
basket['ALARM CLOCK BAKELIKE RED'].sum()

316.0

What is also interesting is to see how the combinations vary by country of purchase. Let’s check out what some popular combinations might be in Germany:

In [12]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

,antecedants,consequents,support,confidence,lift
0,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.584906,4.242887
7,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.571429,4.145125
11,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.843750,6.648168
